In [ ]:
system_prompt ='''
请根据用户提供的自然语言描述生成一个MongoDB的模糊查询语句。
用户输入的内容放置在<input></input>标签中，根据以下步骤生成查询语句：
1. 首先识别用户是否是询问了关于某个问题，如“查找项目名称包含‘***’的记录”、“查找可能设备名称‘****’的信息”等，并提取关键词。
2. 如果用户提出的不是相关查询信息，则询问用户是否需要进一步说明，如“请问您要查找哪个项目的设备信息？”、“请问您要查找哪个设备的采购信息？”等。
3. 如果识别到关键字，将关键字进行提取并按照<requirements></requirements>中的要求输出符合mongodb查询语法的查询语句，最后将完整的可执行查询代码输出在<query></query>中。
<requirements>
目标集合的名称为`equipment_collection`包含以下字段：`project_name`（字符串，项目名称）、`contract_number`（字符串，合同编号）、`contract_type`（字符串，合同类型）、\
`subitem_name`（字符串，子项名称）、`device_name`（字符串，设备名称）、`specification_material`（字符串，规格材质）、\
`manufacturer`（字符串，制造商）。\
用户的需求是：查找与输入关键词相关的所有记录，关键词可能出现在上述任意字段中。
查询需要支持模糊匹配（大小写不敏感），并尽可能覆盖所有可能的数据，查询结果包含所有字段。
注意不要虚构任何数据并按照用户的需求进行查询。
请提供完整的MongoDB查询代码，并确保语法正确。
</requirements>
'''

In [11]:
from ollama import Client
client = Client(
  host='http://192.168.43.41:11434',
  headers={'x-some-header': 'some-value'}
)
response = client.chat(model='qwq:latest', 
                       options={
                           'temperature':0,
                           "num_ctx": 4096,
                       },
                       messages=[
  {'role':'system', 'content': system_prompt},
  {
    'role': 'user',
    'content': f"<input>宝钢德盛不锈钢有限公司烧结余热项目采购了哪些设备？</input>"+"<think>\n",
  },
])
print(response.message.content)

input_text = response.message.content

<think>
好的，我现在需要处理用户的这个查询：“宝钢德盛不锈钢有限公司烧结余热项目采购了哪些设备？”首先，我要确定用户的需求是什么。他们想知道这个特定项目下采购的设备信息。

根据要求，我得先分析输入内容中的关键词。这里的关键显然有两个部分：公司名称“宝钢德盛不锈钢有限公司”和项目名称“烧结余热项目”。不过项目名称可能在数据库里的字段是project_name，所以需要把这两个结合起来作为搜索条件吗？或者用户提到的整个短语其实是完整的项目名称？

接下来，我需要检查用户的问题是否符合查询需求。用户是在询问某个项目的设备采购情况，因此属于明确的查询请求，不需要进一步询问。

现在要提取关键词。这里的主要关键词应该是“宝钢德盛不锈钢有限公司烧结余热项目”，但可能在project_name字段中存储的是完整的项目名称。或者公司名和项目名是分开的部分？比如，可能project_name是“烧结余热项目”，而合同或子项里有公司信息？不过根据问题描述中的输入示例，用户直接提到了整个公司加项目的名称，所以应该将整个短语作为project_name的模糊匹配条件。

另外，用户问的是采购了哪些设备，因此需要查询所有相关的记录，并返回device_name等字段的信息。但根据要求，只要生成查询语句即可，不需要处理结果展示。

接下来要考虑MongoDB的查询语法。因为要模糊匹配且不区分大小写，所以应该使用正则表达式。在MongoDB中，可以用$regex来实现，并设置$options为'i'表示忽略大小写。

因此，构建查询条件时，project_name字段需要包含用户提供的整个项目名称字符串。可能用户的输入中的公司名和项目名是连在一起的，所以在project_name里应该匹配这个完整的字符串。所以正则表达式应该是/宝钢德盛不锈钢有限公司烧结余热项目/i，并且使用$regex来匹配。

然后，查询语句需要覆盖所有相关字段吗？根据要求，关键词可能出现在任意字段中，但用户的问题明确是关于某个项目的设备采购，因此主要关注project_name是否匹配。不过题目中的要求指出“关键词可能出现在上述任意字段中”，所以是否要同时检查其他字段？

仔细看用户的输入：“宝钢德盛不锈钢有限公司烧结余热项目采购了哪些设备？”这里的关键词可能指的是整个项目名称，而用户想知道这个项目的设备信息。因